You should perform some meaningful analysis of the data, that includes at least the following types of operation:

1. Select rows using string methods (e.g. .isin) and numerical conditions

2. Sort table using one or more columns

3. Group and aggregate rows with "groupby"

4. Create a new variable with "apply" and "lambda"

### Determine average loan based on gender in each country within the food and agriculture sectors

In [1]:
import pandas as pd
import requests

In [2]:
# Load data
df = pd.read_csv('kiva_loans.csv')

In [3]:
# Filter out values that don't have 'male' or 'female' as values in the borrower_genders column
filtered_df = df[df['borrower_genders'].isin(['male', 'female'])]

In [4]:
# How many unique currencies
currencies = df['currency'].unique()
sorted_currencies = sorted(currencies)

In [5]:
# Select rows using string methods (e.g. .isin) and numerical conditions
# Select rows where 'sector' is food or ag
sector_filter = filtered_df[filtered_df['sector'].str.contains('Agriculture') | filtered_df['sector'].str.contains('Food')].copy()

In [6]:
# Convert all currency to EUR to standardize
# Get data from currency API
url = 'https://api.exchangerate-api.com/v4/latest/EUR'
response = requests.get(url)
currency_data = response.json()

In [7]:
# Check that the values from the currency_data dictionary are in the sorted df by adding a currency_matches column
sector_filter['currency_matches'] = sector_filter['currency'].isin(currency_data['rates'])

In [8]:
# Remove rows where currency is 'ALL' 
sector_filter = sector_filter.loc[sector_filter['currency'] != 'ALL'].copy()

In [9]:
# Add a column with the currency converted to EUR
sector_filter['loan_amount_eur'] = sector_filter.apply(
    lambda row: row['loan_amount'] / currency_data['rates'].get(row['currency'], 1),  # Default rate is 1 if currency not found
    axis=1
)

In [10]:
# Group and aggregate rows with "groupby"
# Group by gender and country
summary_data = sector_filter.groupby(['country', 'borrower_genders', 'loan_amount', 'currency'])['loan_amount_eur'].mean()

In [11]:
# Convert summary data to a df
summary_data_df = summary_data.reset_index()
# Round loan_amount_eur
summary_data_df.loan_amount_eur = summary_data_df.loan_amount_eur.round(3)

In [12]:
# Sort data by highest loans
sorted_summary = summary_data_df.sort_values(by='loan_amount_eur', ascending=False)